In [1]:
import pandas as pd 
import requests
import json

# Request Tokens

In [2]:
# Read in secret key from json file
secret_key_file  = open("../spotify/secret/secret_key.json")
secret_key_json = json.load(secret_key_file)
secret_key = secret_key_json.get("secret_key")
secret_key_file.close()

#create the get_token function to ask spotify api for an access token
def get_token():
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": f"Basic {secret_key}",
        "Content-Type" : "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type" : "client_credentials"
    }
    response = requests.post(url, data = data, headers = headers)
    
    return response.json()


# test Get Available Genre Seeds

**Sample Requests**
curl --request GET \
  --url https://api.spotify.com/v1/recommendations/available-genre-seeds \
  --header 'Authorization: ' \
  --header 'Content-Type: application/json'Get Available Genre Seeds

In [3]:
foo = get_token()
foo

{'access_token': 'BQDVu_-Sic9FniHAVwjn6E2bZ_1nQ5o0zKUUiSZRjdvDC012ZX5HKF0u7-mEtPO2IMQo91wK7mQOB7kKVWSaxVDxiXi0UYA_BRWe55iMPMfvvln5U80',
 'token_type': 'Bearer',
 'expires_in': 3600}

In [4]:
def get_available_genre_seeds():
    #define base url
    url = "https://api.spotify.com/v1/recommendations/available-genre-seeds"

    #get token to put in headers
    access_token = get_token().get("access_token")
    
    #define headers
    headers = {
        "Authorization" : f"Bearer {access_token}" ,
        "Content-Type" : "application/json"
    }

    #ask spotify for data
    response = requests.get(url, headers = headers)

    #return data
    return response.json()
    


### Create a genre saver function just in case we need to read in genres again

In [5]:
def save_genres():
    #get genre seeds from spotify
    genre_seeds = get_available_genre_seeds().get("genres")

    #write seeds to file
    with open("../spotify/data/genre_seeds.txt", "w") as fp:
        for item in genre_seeds:
            fp.write("%s\n" % item)
        print("Done")
    


function to read in genre seed list each time the notebook is rerun,
the genre seeds are static so there is no point in re-asking spotify for the data
on each notebook run

In [6]:
### Returns a list of genres that are are valid entries for recomendation endpoints ### 

def read_genres():
    with open("../spotify/data/genre_seeds.txt", "r") as fp:
        genres = [line[:-1] for line in fp]
    print("Done")
    
    return genres 
genre_seeds = read_genres()

Done


# Next Section

next steps:
1. keep reading documentation to see what we can bring in and generate project ideas
2. less important - investigate what we can acheive in the recomendation endpoint with genre seeds

## Endpoint to investigate
1. Ad targeting
2. get new releases
3. Audio stats for tracks
4. pulling in my own listening data (users top music endpoint)

# Plug in genre into recommendations to get songs

**dummy query test**

In [51]:
def dummy_recommendations(track_id = "4nFAL2TKUOoAPQJ5DGGoTd"):
   # url = "https://api.spotify.com/v1/recommendations?seed_artists=4NHQUGzhtTLFvgF5SZesLK&seed_genres=classical,country&seed_tracks=0c6xIDDpzE81m2q797ordA"
    
    #url = "https://api.spotify.com/v1/recommendations?seed_genres=alt-rock,alternative,folk,rock"

    url = f"https://api.spotify.com/v1/recommendations?seed_tracks={track_id}"

    access_token = get_token().get('access_token')

    headers = {
        "Content-Type" : "application/json",
        "Authorization" : f"Bearer {access_token}"
    }

    response = requests.get(url, headers = headers)

    return response.json()


Recommendations endpoint works, this endpoint function will forever be known as dummy recommendations
you can input just genres,
or a combination of genres, songs, and artists as seeds.
there is also a limit filter min = 1 max = 100

# Testing Search Endpoint

- search endpoint can return new music with the tag:new filter; which will return music released in the past two weeks
- basic searches such as album, artists, track, year, and genre
- tag:new will return albumns released in the past two weeks
- **tag hipster will return albums with the loweset 10% popularity**

**Requires type in request; type=album track** just specifies what type of music form are we looking for
- also takes a limit filter to say how many results we want 0-50 inclusive
- can use offset to look deeper

In [8]:
def search_endpoint(search_term = "clay%20pigeons", music_type = "track,album"):
    #turn spaces into %20
    if search_term.find(" ") != -1:
        search_term = search_term.replace(' ', '%20')
        
    url = f"https://api.spotify.com/v1/search?q=track:{search_term}&type={music_type}"

    access_token = get_token().get('access_token')
    
    headers = {
        "Content-Type" : "application/json",
        "Authorization" : f"Bearer {access_token}"
    }
    
    response = requests.get(url, headers=headers)

    return response.json()

In [9]:
test = search_endpoint(search_term = "stick season", music_type = "track")

# Testing the new music endpoint, what is the direction of music today?
- vs the direction of music in past ages perhaps.. could be a really cool project

In [10]:
def get_new_releases(country = "US", limit = 20, offset = 0):
    url = f"https://api.spotify.com/v1/browse/new-releases?country={country}&limit={limit}&offset={offset}"

    access_token = get_token().get('access_token')

    headers = {
        'Content-Type' : 'application/json',
        'Authorization' : f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers)

    return response.json()



In [11]:
new_releases = get_new_releases(limit = 5)

In [12]:
new_releases.keys()

dict_keys(['albums'])

In [13]:
new_releases = new_releases.get('albums')

In [14]:
new_releases.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [15]:
len(new_releases.get('items'))

5

In [16]:
new_releases = new_releases.get('items')

In [17]:
sam_hunt_album = new_releases[0]

In [18]:
sam_hunt_album.keys()

dict_keys(['album_type', 'artists', 'available_markets', 'external_urls', 'href', 'id', 'images', 'name', 'release_date', 'release_date_precision', 'total_tracks', 'type', 'uri'])

In [19]:
sam_hunt_album.get('id')

'7txGsnDSqVMoRl6RQ9XyZP'

# Set up Get Album to feed new releases too

In [20]:
def get_album(id = '2T1qRB6zSSGdNgcTD3Dg8H', market = 'US'):
    url = f"https://api.spotify.com/v1/albums/{id}"

    access_token = get_token().get('access_token')

    headers = {
        'Content-Type' : 'application/json',
        'Authorization' : f'Bearer {access_token}'
    }

    response = requests.get(url, headers=headers)

    return response.json()

In [21]:
sam_hunt_album_tracks = get_album()

In [22]:
sam_hunt_album_tracks

{'album_type': 'single',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2kucQ9jQwuD8jWdtR9Ef38'},
   'href': 'https://api.spotify.com/v1/artists/2kucQ9jQwuD8jWdtR9Ef38',
   'id': '2kucQ9jQwuD8jWdtR9Ef38',
   'name': 'Sam Hunt',
   'type': 'artist',
   'uri': 'spotify:artist:2kucQ9jQwuD8jWdtR9Ef38'}],
 'available_markets': ['AD',
  'AE',
  'AG',
  'AL',
  'AM',
  'AO',
  'AR',
  'AT',
  'AU',
  'AZ',
  'BA',
  'BB',
  'BD',
  'BE',
  'BF',
  'BG',
  'BH',
  'BI',
  'BJ',
  'BN',
  'BO',
  'BR',
  'BS',
  'BT',
  'BW',
  'BZ',
  'CA',
  'CD',
  'CG',
  'CH',
  'CI',
  'CL',
  'CM',
  'CO',
  'CR',
  'CV',
  'CW',
  'CY',
  'CZ',
  'DE',
  'DJ',
  'DK',
  'DM',
  'DO',
  'DZ',
  'EC',
  'EE',
  'EG',
  'ES',
  'FI',
  'FJ',
  'FM',
  'FR',
  'GA',
  'GB',
  'GD',
  'GE',
  'GH',
  'GM',
  'GN',
  'GQ',
  'GR',
  'GT',
  'GW',
  'GY',
  'HK',
  'HN',
  'HR',
  'HT',
  'HU',
  'ID',
  'IE',
  'IL',
  'IN',
  'IQ',
  'IS',
  'IT',
  'JM',
  'JO',
  'JP',
  'KE',


# Get Several Tracks Function

In [23]:
#going to pass in a list of ids and unpack them into a string with the ids seperated by a comma , ex. '7ouMYWpwJ422jRcDASZB7P,4VqPOruhp5EdPBeR92t6lQ'
#def get_several_tracks(ids):

IndentationError: expected an indented block (100467330.py, line 2)

# Find recomendations for butchy's song

In [54]:
def get_recs(**kwargs):
    if kwargs.get('mode') == "search":
        search_response = search(search_term = kwargs.get('name'), music_type = 'track')
        rec_id = search_response.get('tracks').get('items')[0].get('id')

        recs = recommendations(track_id=rec_id)
        return recs
        


In [55]:
#test = get_recs(name = "Butchie's Tune", mode='search')

In [57]:
import test
print(test.display_message())

This is the Spotify library file! To make it easier to interact with the spotify API and collect some good data.
